# Westmeyer Lab Presents: A Napari Implementation for Encapsulin Image Analysis

## Imports

In [1]:
import napari
from magicgui import magic_factory
from magicgui.widgets import PushButton, FileEdit
import matplotlib.pyplot as plt
import numpy as np
from skimage.measure import label, regionprops, profile_line
from skimage import exposure, measure, filters
from skimage.draw import disk
from scipy.signal import find_peaks
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
import json
import os
import tifffile
from skimage.draw import polygon
from PIL import Image, ImageDraw
#from plotly.io import to_html
#viewer = napari.view_image(nuclei, colormap='magma')

## Functions

### Extract and filter blobs

In [2]:
# Global variables to store the stacked regions and number of images to select
stacked_regions = None
n_select = 0
# Global variables to store the figure and axes
fig = None
ax = None
canvas = None

@magic_factory(image_layer={'label': 'Select Image Layer'},
               mask_layer={'label': 'Select Mask Layer'})
def test_blob_centers(image_layer: 'napari.layers.Image', mask_layer: 'napari.layers.Image', viewer: 'napari.Viewer') -> None:
    global stacked_regions

    img_data = image_layer.data
    mask_data = mask_layer.data
    img_name = image_layer.name

    labeled_mask = label(mask_data)

    regions = []
    for region in regionprops(labeled_mask):
        centroid_y, centroid_x = map(int, region.centroid)
        min_y, max_y = centroid_y - 25, centroid_y + 25
        min_x, max_x = centroid_x - 25, centroid_x + 25

        region_img = img_data[max(min_y, 0):max_y, max(min_x, 0):max_x]

        if region_img.shape == (50, 50):
            regions.append(region_img)

    if regions:
        stacked_regions = np.array(regions)
        viewer.add_image(stacked_regions, name=f'{img_name}_Regions')

    # Auto-center and zoom
    viewer.camera.center = (stacked_regions.shape[2] // 2, stacked_regions.shape[1] // 2)
    viewer.camera.zoom = 3  # Adjust zoom level as needed

@magic_factory(
    image_layer={'label': 'Select Image Layer'},
    n_select={'label': 'Number of Images to Select', 'min': 1, 'max': 100, 'step': 1}
)
def select_random_images(image_layer: 'napari.layers.Image', n_select: int, viewer: 'napari.Viewer') -> None:
    global stacked_regions

    if stacked_regions is not None:
        img_name = image_layer.name

        # Randomly select n_select images from the stack
        selected_indices = np.random.choice(len(stacked_regions), size=min(n_select, len(stacked_regions)), replace=False)
        selected_images = stacked_regions[selected_indices]

        # Create a grid of selected images
        grid_size = int(np.ceil(np.sqrt(len(selected_images))))
        grid_img = np.zeros((grid_size * 50, grid_size * 50))

        for i, img in enumerate(selected_images):
            row = i // grid_size
            col = i % grid_size
            grid_img[row * 50:(row + 1) * 50, col * 50:(col + 1) * 50] = img

        # Add the grid image as a new layer
        viewer.add_image(grid_img, name=f'{img_name}_SquaresGrid')
        viewer.add_image(selected_images, name=f'{img_name}_RandSq')
    else:
        print("No regions available. Please run the blob center detection first.")

@magic_factory(image_stack={'label': 'Select Image Stack'})
def process_image(image_stack: 'napari.layers.Image', viewer: 'napari.Viewer') -> None:
    global fig, ax, canvas

    img_stack_data = image_stack.data

    radii = []
    img_stack_mean = []

    # Compute radius for each image in the stack
    for img_data in img_stack_data:
        # Normalize the image
        img_normalized = exposure.rescale_intensity(img_data, in_range='image', out_range=(0, 1))
        img_stack_mean.append(img_normalized)

        # Apply Sobel filter
        edges = filters.sobel(img_normalized)

        # Label connected regions
        labeled_edges = measure.label(edges > 0.1)
        regions = measure.regionprops(labeled_edges)

        if regions:
            largest_region = max(regions, key=lambda r: r.area)
            centroid = (largest_region.centroid[0], largest_region.centroid[1])
            rr, cc = disk(centroid, largest_region.equivalent_diameter + 10, shape=edges.shape)
            profile = profile_line(edges, (centroid[0], centroid[1]), (cc[-1], rr[-1]), mode='nearest')
            peaks, _ = find_peaks(profile, distance=10)

            if len(peaks) >= 2:
                peak_distances = np.diff(peaks)
                radius = peak_distances[np.argmax(peak_distances)] / 2
                radii.append(radius)

    mean_radius = np.mean(radii) if radii else 0
    std_radius = np.std(radii) if radii else 0
    print(f"Mean Radius of the detected circles: {mean_radius:.2f} pixels")
    print(f"Standard Deviation of the Radii: {std_radius:.2f} pixels")

    img_mean = np.mean(img_stack_mean, axis=0)
    img_mean_normalized = exposure.rescale_intensity(img_mean, in_range='image', out_range=(0, 1))
    viewer.add_image(img_mean_normalized, name=f'{image_stack.name}_MeanImage')

    edges = filters.sobel(img_mean_normalized)
    labeled_edges = measure.label(edges > 0.1)
    regions = measure.regionprops(labeled_edges)

    if regions:
        largest_region = max(regions, key=lambda r: r.area)
        centroid = (largest_region.centroid[0], largest_region.centroid[1])
        radius = int(largest_region.equivalent_diameter / 2)
        radplusten = radius + 10

        viewer.add_image(edges, name=f'{image_stack.name}_SobelFiltered')

        regions_overlay = np.zeros_like(edges, dtype=np.int32)
        minr, minc, maxr, maxc = largest_region.bbox
        regions_overlay[minr:maxr, minc:maxc] = 1
        viewer.add_labels(regions_overlay, name=f'{image_stack.name}_DetectedCircle')

        rr, cc = disk(centroid, radplusten, shape=img_mean_normalized.shape)
        profile = profile_line(img_mean_normalized, (centroid[0], centroid[1]), (cc[-1], rr[-1]), mode='nearest')

        std_profile = np.zeros_like(profile)
        for i in range(len(profile)):
            std_profile[i] = np.std([profile_line(img, (centroid[0], centroid[1]), (cc[-1], rr[-1]), mode='nearest')[i] for img in img_stack_mean])

        x_values_nm = np.arange(len(profile)) * 2

        if fig is None and ax is None:
            fig, ax = plt.subplots()
            canvas = FigureCanvas(fig)
            viewer.window.add_dock_widget(canvas, name='Radial Profile')

        sample_name = image_stack.name if hasattr(image_stack, 'name') else 'Sample'
        line, = ax.plot(x_values_nm, profile, label=f'{sample_name} - Mean Radius: {mean_radius*2:.2f} nm, Std Dev: {std_radius*2:.2f} nm')
        ax.fill_between(x_values_nm, profile - std_profile, profile + std_profile, color=line.get_color(), alpha=0.5)
        ax.set_title('Radial Plot')
        ax.set_xlabel('Radius (nm)')
        ax.set_ylabel('Intensity')
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles=[handles[0]], labels=[labels[0]])
        canvas.draw()

        save_button = PushButton(label="Save Plot")
        file_edit = FileEdit(label="Save As", mode='w', filter="PNG Files (*.png);;All Files (*)")

        def save_plot():
            file_path = file_edit.value
            if file_path:
                fig.savefig(file_path)

        save_button.changed.connect(save_plot)
        viewer.window.add_dock_widget(save_button, name="Save Plot")
        viewer.window.add_dock_widget(file_edit, name="Save As")
    else:
        print("No regions found.")

@magic_factory(
    src_folder={'label': 'Select source folder with JSON files'},
    output_folder={'label': 'Select output folder'},
)
def generate_masks(src_folder: str, output_folder: str) -> None:
    """
    Generate TIF mask images from JSON files in the source folder and save them to the output folder.
    """
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate over all .json files in the source directory
    for filename in os.listdir(src_folder):
        if filename.endswith('.json'):
            json_path = os.path.join(src_folder, filename)

            # Load the JSON data
            with open(json_path, 'r') as file:
                mask_data = json.load(file)

            # Generate a dummy image_data array with the desired size 1024x768
            image_data = np.zeros((768, 1024), dtype=np.uint8)  # Replace this with actual image loading logic

            # Generate the mask image
            mask = np.zeros_like(image_data, dtype=np.uint8)
            for shape in mask_data['shapes']:
                if shape['shape_type'] == 'polygon':
                    points = np.array(shape['points'], dtype=np.int32)
                    cv2.fillPoly(mask, [points], color=255)

            # Save the mask as a .tif file in the output directory
            output_filename = os.path.splitext(filename)[0] + '_mask.tif'
            output_path = os.path.join(output_folder, output_filename)
            io.imsave(output_path, mask)

    print("Masks generated and saved successfully.")


### Initialize Napari Window and Dock Widgets

In [3]:
# Initialize the viewer and widgets
viewer = napari.Viewer()

# Add the test_blob_centers widget
my_widget1 = test_blob_centers()
dw1 = viewer.window.add_dock_widget(my_widget1, name="Extract Filter and Classify")

# Add the select_random_images widget
my_widget2 = select_random_images()
dw2 = viewer.window.add_dock_widget(my_widget2, name="Select Random Images")

my_widget3 = process_image()
dw3 = viewer.window.add_dock_widget(my_widget3, name="Process Images")

my_widget4 = generate_masks()
dw4 = viewer.window.add_dock_widget(my_widget4, name="Convert Mask: JSON to TIF")

@viewer.bind_key('d')
def delete_current_image(viewer):
    global stacked_regions
    if stacked_regions is not None:
        z_index = int(viewer.dims.point[0])

        # Remove the image at the current z-index
        stacked_regions = np.delete(stacked_regions, z_index, axis=0)

        # Update the viewer with the modified stack
        viewer.layers['Stacked Regions'].data = stacked_regions


/opt/homebrew/Caskroom/miniforge/base/envs/napari-env/lib/python3.12/site-packages/pytools/persistent_dict.py:59: UserWarning: pytools.persistent_dict: unable to import 'siphash24.siphash13', falling back to hashlib.sha256
  warn("pytools.persistent_dict: unable to import 'siphash24.siphash13', "


Mean Radius of the detected circles: 7.20 pixels
Standard Deviation of the Radii: 1.42 pixels
